In [1]:
import pandas as pd
import ast


In [2]:
file_path = '../pred_visualiser/input_tsvs/lear_slog_final_epoch0.tsv'

with open(file_path, 'r') as file:
    lines = file.readlines()

data = []
for line in lines:
    if line.strip():
        # Remove the 'span2semantic' field using string manipulation
        line_mod = line
        # Find the start and end of the 'span2semantic' field
        start = line_mod.find("'span2semantic':")
        if start != -1:
            # Find the comma after the field to remove it cleanly
            end = line_mod.find("},", start)
            if end != -1:
                # Remove the field including the trailing comma
                line_mod = line_mod[:start] + line_mod[end+2:]
            else:
                # If it's the last field, remove up to the closing brace
                end = line_mod.find("}", start)
                line_mod = line_mod[:start] + line_mod[end+1:]
        # Parse the modified line
        data.append(ast.literal_eval(line_mod))

df = pd.DataFrame(data)


In [3]:
df.head()  # Display the first few rows of the DataFrame

,bottom_span,span2output_token,parent_child_spans,end_span,pred_chain,label_chain,pair,parent_json,category
0,"[[9, 9], [7, 7], [5, 5], [4, 4], [1, 1]]","[[[1, 1], cake], [[4, 4], cat], [[5, 5], like]...","[[[4, 5], [[4, 4], [5, 5]]], [[7, 9], [[7, 7],...","[0, 9]","[like, cat, cake, None, xcomp, eat, cat, None,...","[nmod, like, cat, cake, None, eat, elizabeth, ...",[the cake that a cat liked was eaten by elizab...,"{'children': [{'children': [], 'token': 'cat',...",RC_modif_subj
1,"[[21, 21], [18, 18], [16, 16], [14, 14], [11, ...","[[[1, 1], bird], [[4, 4], room], [[5, 5], like...","[[[11, 14], [[11, 11], [14, 14]]], [[1, 4], [[...","[0, 21]","[like, bird, in, room, None, None, ccomp, mail...","[like, bird, in, room, None, None, ccomp, mail...",[a bird in the room liked that a girl in the t...,"{'children': [{'children': [{'children': [], '...",PP_modif_subj
2,"[[7, 7], [6, 6], [3, 3], [0, 0]]","[[[0, 0], who], [[3, 3], teacher], [[6, 6], sh...","[[[6, 7], [[6, 6], [7, 7]]], [[3, 7], [[3, 3],...","[0, 8]","[grow, shoe, teacher, who]","[grow, teacher, beside, shoe, who, None]","[who did the teacher beside the shoe grow ?, *...","{'children': [{'children': [], 'token': 'shoe'...",Q_modified_NPs
3,"[[18, 18], [15, 15], [13, 13], [10, 10], [7, 7...","[[[1, 1], dog], [[2, 2], prove], [[4, 4], char...","[[[1, 2], [[1, 1], [2, 2]]], [[15, 18], [[15, ...","[0, 18]","[prove, dog, None, None, ccomp, give, charlott...","[prove, dog, None, None, ccomp, give, charlott...",[a dog proved that charlotte gave the servant ...,"{'children': [{'children': [], 'token': 'dog',...",PP_modif_iobj
4,"[[7, 7], [6, 6], [4, 4], [3, 3], [1, 1]]","[[[1, 1], mouse], [[3, 3], send], [[4, 4], emm...","[[[6, 7], [[6, 6], [7, 7]]], [[4, 7], [[4, 4],...","[0, 7]","[send, None, mouse, None, ccomp, burn, donut, ...","[nmod, send, mouse, donut, emma, burn, None, m...","[a mouse that sent emma a donut burned, mouse ...","{'children': [{'children': [{'children': [], '...",RC_modif_subj


In [4]:
# drop all rows except pred_chain, label_chain, pair, and category
df = df[['pred_chain', 'label_chain', 'pair', 'category']]

In [5]:
# filter for category "RC_modif_iobj"
df_rc_modif_iobj = df[df['category'] == 'RC_modif_iobj']
df_rc_modif_subj = df[df['category'] == 'RC_modif_subj']
df_center_embed_3 = df[df['category'] == 'center_embed_3']
df_center_embed_5_12 = df[df['category'] == 'center_embed_5-12']
df_RC_iobj_extracted = df[df['category'] == 'RC_iobj_extracted']

df_rc_modif_iobj.head()  # Display the first few rows of the filtered DataFrame

,pred_chain,label_chain,pair,category
9,"[give, None, cake, girl, ccomp, intend, None, ...","[give, None, cake, girl, nmod, intend, girl, N...",[a cake was given to the girl that intended to...,RC_modif_iobj
30,"[give, visitor, cookie, girl, ccomp, roll, Non...","[give, visitor, cookie, girl, nmod, roll, emma...",[a visitor gave a cookie to the girl that emma...,RC_modif_iobj
34,"[hope, girl, None, None, ccomp, lend, olivia, ...","[hope, girl, None, None, ccomp, lend, olivia, ...",[a girl hoped that olivia lent a cake to the f...,RC_modif_iobj
53,"[give, emma, girl, None, ccomp, award, None, c...","[give, emma, shoe, girl, nmod, award, girl, ch...",[emma gave the girl that awarded the teacher a...,RC_modif_iobj
76,"[give, avery, girl, None, xcomp, eat, avery, f...","[give, avery, flower, girl, nmod, eat, scarlet...",[avery gave a girl that scarlett ate the flowe...,RC_modif_iobj


In [6]:
def to_list(x):
    if isinstance(x, str):
        return ast.literal_eval(x)
    return x

for df_cat in [df_rc_modif_subj, df_rc_modif_iobj]:
    df_cat['pred_chain'] = df_cat['pred_chain'].apply(to_list)
    df_cat['label_chain'] = df_cat['label_chain'].apply(to_list)

def move_nmod(chain):
    if isinstance(chain, list) and len(chain) > 4 and chain[0] == 'nmod':
        # Remove 'nmod' from the first position and insert at the fourth
        chain = chain[1:]
        chain.insert(4, 'nmod')
    return chain

for df_cat in [df_rc_modif_subj, df_rc_modif_iobj, df_center_embed_3, df_center_embed_5_12, df_RC_iobj_extracted]:
    df_cat['label_chain'] = df_cat['label_chain'].apply(move_nmod)

C:\Users\joaqu\AppData\Local\Temp\ipykernel_15460\3098330883.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['pred_chain'] = df_cat['pred_chain'].apply(to_list)
C:\Users\joaqu\AppData\Local\Temp\ipykernel_15460\3098330883.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['label_chain'] = df_cat['label_chain'].apply(to_list)
C:\Users\joaqu\AppData\Local\Temp\ipykernel_15460\3098330883.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [7]:
df_rc_modif_subj.head()

,pred_chain,label_chain,pair,category
0,"[like, cat, cake, None, xcomp, eat, cat, None,...","[like, cat, cake, None, nmod, eat, elizabeth, ...",[the cake that a cat liked was eaten by elizab...,RC_modif_subj
4,"[send, None, mouse, None, ccomp, burn, donut, ...","[send, mouse, donut, emma, nmod, burn, None, m...","[a mouse that sent emma a donut burned, mouse ...",RC_modif_subj
15,"[feed, None, hat, None, xcomp, sell, None, gir...","[feed, None, hat, girl, nmod, sell, None, hat,...",[a hat that was fed to the girl was sold to no...,RC_modif_subj
43,"[find, emma, dog, None, ccomp, nurse, donut, N...","[find, dog, donut, None, nmod, nurse, dog, emm...","[the dog that found the donut nursed emma, * d...",RC_modif_subj
63,"[slide, emma, teacher, oliver, ccomp, give, No...","[slide, emma, teacher, None, nmod, give, teach...",[the teacher that emma slid gave a game to oli...,RC_modif_subj


In [8]:
df_rc_modif_iobj.head()


,pred_chain,label_chain,pair,category
9,"[give, None, cake, girl, ccomp, intend, None, ...","[give, None, cake, girl, nmod, intend, girl, N...",[a cake was given to the girl that intended to...,RC_modif_iobj
30,"[give, visitor, cookie, girl, ccomp, roll, Non...","[give, visitor, cookie, girl, nmod, roll, emma...",[a visitor gave a cookie to the girl that emma...,RC_modif_iobj
34,"[hope, girl, None, None, ccomp, lend, olivia, ...","[hope, girl, None, None, ccomp, lend, olivia, ...",[a girl hoped that olivia lent a cake to the f...,RC_modif_iobj
53,"[give, emma, girl, None, ccomp, award, None, c...","[give, emma, shoe, girl, nmod, award, girl, ch...",[emma gave the girl that awarded the teacher a...,RC_modif_iobj
76,"[give, avery, girl, None, xcomp, eat, avery, f...","[give, avery, flower, girl, nmod, eat, scarlet...",[avery gave a girl that scarlett ate the flowe...,RC_modif_iobj


In [9]:
# calculate filtered mismatches
def calculate_filtered_mismatches(row):
    pred_chain = row['pred_chain']
    label_chain = row['label_chain']
    if isinstance(pred_chain, list) and isinstance(label_chain, list):
        mismatches = 0
        for p, l in zip(pred_chain, label_chain):
            if p != l:
                # skip allowed mismatches
                if (p == "None") or (p in ["ccomp", "xcomp"] and l == "nmod"):
                    continue
                mismatches += 1
        return mismatches

df_rc_modif_iobj['mismatches'] = df_rc_modif_iobj.apply(calculate_filtered_mismatches, axis=1)
df_rc_modif_subj['mismatches'] = df_rc_modif_subj.apply(calculate_filtered_mismatches, axis=1)
df_RC_iobj_extracted['mismatches'] = df_RC_iobj_extracted.apply(calculate_filtered_mismatches, axis=1)

C:\Users\joaqu\AppData\Local\Temp\ipykernel_15460\4138489958.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rc_modif_iobj['mismatches'] = df_rc_modif_iobj.apply(calculate_filtered_mismatches, axis=1)
C:\Users\joaqu\AppData\Local\Temp\ipykernel_15460\4138489958.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rc_modif_subj['mismatches'] = df_rc_modif_subj.apply(calculate_filtered_mismatches, axis=1)
C:\Users\joaqu\AppData\Local\Temp\ipykernel_15460\4138489958.py:17: SettingWithCopyWarning: 
A

In [10]:
df_rc_modif_iobj.head()

,pred_chain,label_chain,pair,category,mismatches
9,"[give, None, cake, girl, ccomp, intend, None, ...","[give, None, cake, girl, nmod, intend, girl, N...",[a cake was given to the girl that intended to...,RC_modif_iobj,0
30,"[give, visitor, cookie, girl, ccomp, roll, Non...","[give, visitor, cookie, girl, nmod, roll, emma...",[a visitor gave a cookie to the girl that emma...,RC_modif_iobj,1
34,"[hope, girl, None, None, ccomp, lend, olivia, ...","[hope, girl, None, None, ccomp, lend, olivia, ...",[a girl hoped that olivia lent a cake to the f...,RC_modif_iobj,0
53,"[give, emma, girl, None, ccomp, award, None, c...","[give, emma, shoe, girl, nmod, award, girl, ch...",[emma gave the girl that awarded the teacher a...,RC_modif_iobj,1
76,"[give, avery, girl, None, xcomp, eat, avery, f...","[give, avery, flower, girl, nmod, eat, scarlet...",[avery gave a girl that scarlett ate the flowe...,RC_modif_iobj,3


In [11]:
df_rc_modif_subj.head()

,pred_chain,label_chain,pair,category,mismatches
0,"[like, cat, cake, None, xcomp, eat, cat, None,...","[like, cat, cake, None, nmod, eat, elizabeth, ...",[the cake that a cat liked was eaten by elizab...,RC_modif_subj,1
4,"[send, None, mouse, None, ccomp, burn, donut, ...","[send, mouse, donut, emma, nmod, burn, None, m...","[a mouse that sent emma a donut burned, mouse ...",RC_modif_subj,3
15,"[feed, None, hat, None, xcomp, sell, None, gir...","[feed, None, hat, girl, nmod, sell, None, hat,...",[a hat that was fed to the girl was sold to no...,RC_modif_subj,1
43,"[find, emma, dog, None, ccomp, nurse, donut, N...","[find, dog, donut, None, nmod, nurse, dog, emm...","[the dog that found the donut nursed emma, * d...",RC_modif_subj,3
63,"[slide, emma, teacher, oliver, ccomp, give, No...","[slide, emma, teacher, None, nmod, give, teach...",[the teacher that emma slid gave a game to oli...,RC_modif_subj,1


In [12]:
counts = df_rc_modif_iobj['mismatches'].value_counts().sort_index()
percentages = (counts / len(df_rc_modif_iobj) * 100).round(2)
print("\nPercentages (% of total):\n", percentages)
print(f"Total rows: {len(df_rc_modif_iobj)}")


Percentages (% of total):
 mismatches
0    53.6
1    12.1
2    17.6
3    13.4
4     3.3
Name: count, dtype: float64
Total rows: 1000


In [13]:
counts_subj = df_rc_modif_subj['mismatches'].value_counts().sort_index()
percentages_subj = (counts_subj / len(df_rc_modif_subj) * 100).round(2)
print("\nPercentages (% of total):\n", percentages_subj)
print(f"Total rows: {len(df_rc_modif_subj)}")


Percentages (% of total):
 mismatches
0    34.5
1    37.5
2    14.2
3     7.6
4     3.5
5     1.9
6     0.6
7     0.2
Name: count, dtype: float64
Total rows: 1000


In [14]:
# .head() examples of rows with more than 2 mismatches from df_rc_modif_iobj
df_rc_modif_iobj[df_rc_modif_iobj['mismatches'] > 0].head()

,pred_chain,label_chain,pair,category,mismatches
30,"[give, visitor, cookie, girl, ccomp, roll, Non...","[give, visitor, cookie, girl, nmod, roll, emma...",[a visitor gave a cookie to the girl that emma...,RC_modif_iobj,1
53,"[give, emma, girl, None, ccomp, award, None, c...","[give, emma, shoe, girl, nmod, award, girl, ch...",[emma gave the girl that awarded the teacher a...,RC_modif_iobj,1
76,"[give, avery, girl, None, xcomp, eat, avery, f...","[give, avery, flower, girl, nmod, eat, scarlet...",[avery gave a girl that scarlett ate the flowe...,RC_modif_iobj,3
84,"[serve, monkey, cat, None, ccomp, paint, None,...","[serve, monkey, cookie, cat, nmod, paint, cat,...",[a monkey served a cat that painted emma a coo...,RC_modif_iobj,3
126,"[promise, dog, boy, None, ccomp, like, noah, c...","[promise, dog, cake, boy, nmod, like, noah, bo...",[the dog promised a boy that noah liked the ca...,RC_modif_iobj,2


In [15]:
df_rc_modif_subj[df_rc_modif_subj['mismatches'] > 0].head()

,pred_chain,label_chain,pair,category,mismatches
0,"[like, cat, cake, None, xcomp, eat, cat, None,...","[like, cat, cake, None, nmod, eat, elizabeth, ...",[the cake that a cat liked was eaten by elizab...,RC_modif_subj,1
4,"[send, None, mouse, None, ccomp, burn, donut, ...","[send, mouse, donut, emma, nmod, burn, None, m...","[a mouse that sent emma a donut burned, mouse ...",RC_modif_subj,3
15,"[feed, None, hat, None, xcomp, sell, None, gir...","[feed, None, hat, girl, nmod, sell, None, hat,...",[a hat that was fed to the girl was sold to no...,RC_modif_subj,1
43,"[find, emma, dog, None, ccomp, nurse, donut, N...","[find, dog, donut, None, nmod, nurse, dog, emm...","[the dog that found the donut nursed emma, * d...",RC_modif_subj,3
63,"[slide, emma, teacher, oliver, ccomp, give, No...","[slide, emma, teacher, None, nmod, give, teach...",[the teacher that emma slid gave a game to oli...,RC_modif_subj,1


In [16]:
# calculate number of mismatches in first four elements of pred_chain
def calculate_first_pred_mismatches(row):
    pred_chain = row['pred_chain']
    label_chain = row['label_chain']
    if isinstance(pred_chain, list) and isinstance(label_chain, list):
        mismatches = 0
        for p, l in zip(pred_chain[:4], label_chain[:4]):
            if p != l:
                # skip allowed mismatches
                if (p == "None") or (p in ["ccomp", "xcomp"] and l == "nmod"):
                    continue
                mismatches += 1
        return mismatches
    
def calculate_bad_recipient(row):
    pred_chain = row['pred_chain']
    label_chain = row['label_chain']
    if isinstance(pred_chain, list) and isinstance(label_chain, list):
        if label_chain[3] == "None" and pred_chain[3] != "None":
            return "false positive"
        elif label_chain[3] != "None" and pred_chain[3] == "None":
            return "false negative"
        elif label_chain[3] == "None" and pred_chain[3] == "None":
            return "correct negative"
        elif label_chain[3] != "None" and pred_chain[3] != "None":
            return "correct positive"
        else:
            return "unknown"

df_center_embed_3['mismatches'] = df_center_embed_3.apply(calculate_first_pred_mismatches, axis=1)
df_center_embed_5_12['mismatches'] = df_center_embed_5_12.apply(calculate_first_pred_mismatches, axis=1)
df_center_embed_3['bad_recipient'] = df_center_embed_3.apply(calculate_bad_recipient, axis=1)
df_center_embed_5_12['bad_recipient'] = df_center_embed_5_12.apply(calculate_bad_recipient, axis=1)

C:\Users\joaqu\AppData\Local\Temp\ipykernel_15460\500873775.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_center_embed_3['mismatches'] = df_center_embed_3.apply(calculate_first_pred_mismatches, axis=1)
C:\Users\joaqu\AppData\Local\Temp\ipykernel_15460\500873775.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_center_embed_5_12['mismatches'] = df_center_embed_5_12.apply(calculate_first_pred_mismatches, axis=1)
C:\Users\joaqu\AppData\Local\Temp\ipykernel_15460\500873775.py:32: SettingWithCopy

In [17]:
df_center_embed_3[df_center_embed_3['mismatches'] > 0].head()

,pred_chain,label_chain,pair,category,mismatches,bad_recipient
45,"[like, kitty, girl, wolf, ccomp, roll, emma, b...","[like, kitty, girl, None, nmod, roll, emma, bo...",[the kitty liked a girl that the wolf that the...,center_embed_3,1,false positive
58,"[admire, cat, cake, baby, ccomp, like, sofia, ...","[admire, cat, cake, None, nmod, like, sofia, t...",[a cat admired the cake that a baby that a tea...,center_embed_3,1,false positive
59,"[eat, teacher, host, girl, ccomp, freeze, alex...","[eat, teacher, host, None, nmod, freeze, alexa...",[a teacher ate a host that the girl that a bun...,center_embed_3,1,false positive
88,"[slide, joseph, baby, visitor, ccomp, help, gi...","[slide, joseph, baby, None, nmod, help, giraff...",[joseph slid the baby that a visitor that a ch...,center_embed_3,1,false positive
122,"[eat, olivia, cookie, teacher, ccomp, find, gi...","[eat, olivia, cookie, None, nmod, find, girl, ...",[olivia ate a cookie that a teacher that a cat...,center_embed_3,1,false positive


In [18]:
counts_center_embed_3 = df_center_embed_3['mismatches'].value_counts().sort_index()
percentages_center_embed_3 = (counts_center_embed_3 / len(df_center_embed_3) * 100).round(2)
print("\nPercentages (% of total):\n", percentages_center_embed_3)
print(f"Total rows: {len(df_center_embed_3)}")


Percentages (% of total):
 mismatches
0    40.7
1    59.1
2     0.2
Name: count, dtype: float64
Total rows: 1000


In [19]:
counts_center_embed_5_12 = df_center_embed_5_12['mismatches'].value_counts().sort_index()
percentages_center_embed_5_12 = (counts_center_embed_5_12 / len(df_center_embed_5_12) * 100).round(2)
print("\nPercentages (% of total):\n", percentages_center_embed_5_12)
print(f"Total rows: {len(df_center_embed_5_12)}")


Percentages (% of total):
 mismatches
0    16.9
1    66.9
2    16.2
Name: count, dtype: float64
Total rows: 1000


In [20]:
counts_center_embed_3 = df_center_embed_3['bad_recipient'].value_counts().sort_index()
percentages_center_embed_3 = (counts_center_embed_3 / len(df_center_embed_3) * 100).round(2)
print("\nPercentages (% of total):\n", percentages_center_embed_3)
print(f"Total rows: {len(df_center_embed_3)}")


Percentages (% of total):
 bad_recipient
correct negative    40.7
false positive      59.3
Name: count, dtype: float64
Total rows: 1000


In [21]:
counts_center_embed_5_12 = df_center_embed_5_12['bad_recipient'].value_counts().sort_index()
percentages_center_embed_5_12 = (counts_center_embed_5_12 / len(df_center_embed_5_12) * 100).round(2)
print("\nPercentages (% of total):\n", percentages_center_embed_5_12)
print(f"Total rows: {len(df_center_embed_5_12)}")


Percentages (% of total):
 bad_recipient
correct negative    16.9
false positive      83.1
Name: count, dtype: float64
Total rows: 1000


In [22]:
df_RC_iobj_extracted[df_RC_iobj_extracted['mismatches'] == 0].head()

,pred_chain,label_chain,pair,category,mismatches
20,"[dream, sofia, None, None, ccomp, help, lawyer...","[dream, sofia, None, None, ccomp, help, lawyer...",[sofia dreamed that the lawyer helped a girl t...,RC_iobj_extracted,0
118,"[roll, boy, cow, None, ccomp, give, girl, cake...","[roll, boy, cow, None, nmod, give, girl, cake,...",[the boy rolled a cow that a girl gave a cake ...,RC_iobj_extracted,0
218,"[nurse, bee, child, None, ccomp, give, turkey,...","[nurse, bee, child, None, nmod, give, turkey, ...",[the bee nursed a child that the turkey gave a...,RC_iobj_extracted,0
232,"[like, william, girl, None, ccomp, give, boy, ...","[like, william, girl, None, nmod, give, boy, s...",[william liked a girl that a boy gave a sandwi...,RC_iobj_extracted,0
283,"[like, consumer, None, None, ccomp, roll, squi...","[like, consumer, None, None, ccomp, roll, squi...",[the consumer liked that a squirrel rolled the...,RC_iobj_extracted,0


In [23]:
counts_RC_iobj_extracted = df_RC_iobj_extracted['mismatches'].value_counts().sort_index()
percentages_RC_iobj_extracted = (counts_RC_iobj_extracted / len(df_RC_iobj_extracted) * 100).round(2)
print("\nPercentages (% of total):\n", percentages_RC_iobj_extracted)
print(f"Total rows: {len(df_RC_iobj_extracted)}")


Percentages (% of total):
 mismatches
0    25.3
1    74.7
Name: count, dtype: float64
Total rows: 1000


In [27]:
# print all categories and their accuracies (accuracy = proportion of rows with pred_chain == label_chain)
categories = df['category'].unique()
for category in categories:
    df_cat = df[df['category'] == category]
    n_total = len(df_cat)
    n_correct = (df_cat['pred_chain'] == df_cat['label_chain']).sum()
    accuracy = n_correct / n_total if n_total > 0 else float('nan')
    print(f"Category: {category}, Accuracy: {accuracy:.4f} ({n_correct}/{n_total})")

Category: RC_modif_subj, Accuracy: 0.0000 (0/1000)
Category: PP_modif_subj, Accuracy: 0.6690 (669/1000)
Category: Q_modified_NPs, Accuracy: 0.3800 (380/1000)
Category: PP_modif_iobj, Accuracy: 0.6740 (674/1000)
Category: CP_5-12, Accuracy: 1.0000 (1000/1000)
Category: Q_iobj_ditransV, Accuracy: 1.0000 (1000/1000)
Category: center_embed_5-12, Accuracy: 0.0000 (0/1000)
Category: RC_modif_iobj, Accuracy: 0.0000 (0/1000)
Category: Q_dobj_ditransV, Accuracy: 0.4870 (487/1000)
Category: RC_iobj_extracted, Accuracy: 0.0000 (0/1000)
Category: PP_5-12, Accuracy: 1.0000 (1000/1000)
Category: Q_long_mv, Accuracy: 0.0000 (0/1000)
Category: Q_subj_active, Accuracy: 1.0000 (1000/1000)
Category: CP_3, Accuracy: 1.0000 (1000/1000)
Category: center_embed_3, Accuracy: 0.0000 (0/1000)
Category: PP_3, Accuracy: 1.0000 (1000/1000)
Category: Q_subj_passive, Accuracy: 1.0000 (1000/1000)


In [28]:
#print overall accuracy
n_correct = (df['pred_chain'] == df['label_chain']).sum()
n_total = len(df)
overall_accuracy = n_correct / n_total if n_total > 0 else float('nan')
print(f"Overall Accuracy: {overall_accuracy:.4f} ({n_correct}/{n_total})")

Overall Accuracy: 0.5418 (9210/17000)
